Import and setup some auxiliary functions

In [0]:
import torch
from torch import nn, optim
from torchvision import transforms, datasets
from torch.autograd import Variable
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat
from tqdm import tqdm

torch.multiprocessing.set_sharing_strategy('file_system')

def compute_score(acc, min_thres, max_thres):
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
                     * 100
    return base_score


def run(algorithm, dataset_name, filename):
    start = timeit.default_timer()
    predicted_test_labels, gt_labels = algorithm(dataset_name)
    if predicted_test_labels is None or gt_labels is None:
      return (0, 0, 0)
    stop = timeit.default_timer()
    run_time = stop - start
    
    np.savetxt(filename, np.asarray(predicted_test_labels))

    correct = 0
    total = 0
    for label, prediction in zip(gt_labels, predicted_test_labels):
      total += label.size(0)
      correct += (prediction.cpu().numpy() == label.cpu().numpy()).sum().item()   # assuming your model runs on GPU
      
    accuracy = float(correct) / total
    
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    # generate your predictions for submission.
    from google.colab import files
    files.download(filename)

    return (correct, accuracy, run_time)

TODO: Implement Logistic Regression here

In [0]:
import torch
from torch import nn, optim
from torchvision import transforms, datasets
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat
from tqdm import tqdm

torch.multiprocessing.set_sharing_strategy('file_system')

epochs = 10
batch_size_train = 128
batch_size_test = 1000
learining_rate = 1e-3
lambda_val_MNIST = 5e-5
lambda_val_CIFAR10 = 5e-3


def logistic_regression(dataset_name):

    random_seed = 1
    torch.backends.cudnn.enabled = False
    torch.manual_seed(random_seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    if dataset_name == "MNIST":
        MNIST_training = datasets.MNIST('/MNIST_dataset/', train=True, download=True,
                                transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))]))
        
        MNIST_test = datasets.MNIST('/MNIST_dataset/', train=False, download=True,
                                transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))]))
        
        MNIST_training_dataset, MNIST_validation_dataset = random_split(MNIST_training, [55000, 5000])

        train_loader  = DataLoader(MNIST_training_dataset, batch_size=batch_size_train, shuffle=True)
        validation_loader  = DataLoader(MNIST_validation_dataset, batch_size=batch_size_train, shuffle=True)
        test_loader = DataLoader(MNIST_test, batch_size=batch_size_test, shuffle=True)

        # images.shape = [batch_size,channel=1,height=28,width=28]
        model = nn.Linear(28 * 28, 10).to(device)
        criterion = nn.CrossEntropyLoss()
        # adam better than sgd
        optimizer = optim.Adam(model.parameters(), lr=learining_rate, weight_decay=lambda_val_MNIST)
      
    elif dataset_name == "CIFAR10":
        CIFAR10_training = datasets.CIFAR10("/CIFAR10_dataset/",train=True, download=True,
                                transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
        
        CIFAR10_test = datasets.CIFAR10("/CIFAR10_dataset/",train=False, download=True,
                                transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
        
        CIFAR10_training_dataset, CIFAR10_validation_dataset = random_split(CIFAR10_training, [40000, 10000])

        train_loader  = DataLoader(CIFAR10_training_dataset, batch_size=batch_size_train, shuffle=True, num_workers=2)
        validation_loader  = DataLoader(CIFAR10_validation_dataset, batch_size=batch_size_train, shuffle=True, num_workers=2)
        test_loader = DataLoader(CIFAR10_test, batch_size=batch_size_test, shuffle=True, num_workers=2)

        # images.shape = [batch_size,channel=3,height=32,width=32]
        model = nn.Linear(32 * 32 * 3, 10).to(device)
        criterion = nn.CrossEntropyLoss()
        # adam better than sgd
        optimizer = optim.Adam(model.parameters(), lr=learining_rate, weight_decay=lambda_val_CIFAR10)
        

    # shape of data 
    examples = enumerate(test_loader)
    _, (example_data, _) = next(examples)
    print(example_data.shape)

    # train 
    for epoch in range(1, epochs + 1):    
        model.train()
        for i, (images, labels) in enumerate(train_loader): 
            images = images.to(device) 
            labels = labels.to(device)
            optimizer.zero_grad()
            images = images.view(images.size(0), -1)
            output = model(images)
            # l2_reg = 0
            # for param in model.parameters():
            #     l2_reg += torch.norm(param)
            # loss = criterion(output, labels) + lambda_val * l2_reg
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

        # validation 
        if epoch%5== 0:
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for images, labels in validation_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    images = images.view(images.size(0), -1)
                    output = model(images)
                    _, pred = torch.max(output.data, 1)
                    total += labels.size(0)
                    correct += (pred == labels).sum().item()

            accuracy = 100. * correct / total
            print("{}. epoch: {}. Accuracy: {}.".format(dataset_name, epoch, accuracy))

    # test 
    model.eval()
    out1 = []
    out2 = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            images = images.view(images.size(0), -1)
            output = model(images)
            _, pred = torch.max(output.data, 1)  
            pred = pred.cpu().numpy()
            labels = labels.cpu().numpy()
            out1.append(pred)
            out2.append(labels)

    return torch.tensor(out1), torch.tensor(out2)

Main loop. Run time and total score will be shown below.

In [0]:
def run_on_dataset(dataset_name, filename):
    if dataset_name == "MNIST":
        min_thres = 0.82
        max_thres = 0.92

    elif dataset_name == "CIFAR10":
        min_thres = 0.28
        max_thres = 0.38

    correct_predict, accuracy, run_time = run(logistic_regression, dataset_name, filename)

    score = compute_score(accuracy, min_thres, max_thres)
    result = OrderedDict(correct_predict=correct_predict,
                         accuracy=accuracy, score=score,
                         run_time=run_time)
    return result, score


def main():
    filenames = { "MNIST": "predictions_mnist_zijunwu_1488834.txt", "CIFAR10": "predictions_cifar10_zijunwu_1488834.txt"}
    result_all = OrderedDict()
    score_weights = [0.5, 0.5]
    scores = []
    for dataset_name in ["MNIST","CIFAR10"]:
        result_all[dataset_name], this_score = run_on_dataset(dataset_name, filenames[dataset_name])
        scores.append(this_score)
    total_score = [score * weight for score, weight in zip(scores, score_weights)]
    total_score = np.asarray(total_score).sum().item()
    result_all['total_score'] = total_score
    with open('result.txt', 'w') as f:
        f.writelines(pformat(result_all, indent=4))
    print("\nResult:\n", pformat(result_all, indent=4))


main()

cuda
torch.Size([1000, 1, 28, 28])
MNIST. epoch: 5. Accuracy: 92.0.
